In [1]:
#!pip install -r requirements.txt
import soundfile as sf
import pyaudio
import wave
from openai import OpenAI
import torch
import librosa
import librosa.display
import numpy as np
import noisereduce as nr
import scipy.io.wavfile as wav
import os
import subprocess
import whisper


/Users/akshat/anaconda3/envs/VirtualAI/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""def record_audio(): #Function to record audio
       
    FORMAT = pyaudio.paInt16 # data format
    CHANNELS = 1 # mono, change to 2 if you want stereo
    RATE = 44100 # sample rate
    CHUNK = 1024 # frames per buffer
    RECORD_SECONDS = 25 # change this to the duration you want to record
    WAVE_OUTPUT_FILENAME = "output.wav"

    audio = pyaudio.PyAudio()

    # Start recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
    print("Recording...")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    # Stop recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded data as a WAV file
    with wave.open(WAVE_OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
"""

'def record_audio(): #Function to record audio\n\n    FORMAT = pyaudio.paInt16 # data format\n    CHANNELS = 1 # mono, change to 2 if you want stereo\n    RATE = 44100 # sample rate\n    CHUNK = 1024 # frames per buffer\n    RECORD_SECONDS = 25 # change this to the duration you want to record\n    WAVE_OUTPUT_FILENAME = "output.wav"\n\n    audio = pyaudio.PyAudio()\n\n    # Start recording\n    stream = audio.open(format=FORMAT, channels=CHANNELS,\n                        rate=RATE, input=True,\n                        frames_per_buffer=CHUNK)\n    print("Recording...")\n\n    frames = []\n\n    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):\n        data = stream.read(CHUNK)\n        frames.append(data)\n\n    print("Finished recording.")\n\n    # Stop recording\n    stream.stop_stream()\n    stream.close()\n    audio.terminate()\n\n    # Save the recorded data as a WAV file\n    with wave.open(WAVE_OUTPUT_FILENAME, \'wb\') as wf:\n        wf.setnchannels(CHANNELS)\n        wf

In [3]:

def reduce_background_noise(input_audio_path, output_audio_path):
    """
    Reduces background noise from an audio file and saves the cleaned version.

    Args:
        input_audio_path (str): Path to the input audio file (WAV format recommended).
        output_audio_path (str): Path to save the cleaned audio file.
    """
    try:
        # Load audio file
        rate, data = wav.read(input_audio_path)
        
        # Convert to mono if stereo
        if len(data.shape) > 1:
            data = np.mean(data, axis=1)

        # Apply noise reduction
        reduced_noise = nr.reduce_noise(y=data, sr=rate)

        # Save the processed audio
        wav.write(output_audio_path, rate, reduced_noise.astype(np.int16))
        
        print(f"Noise reduction completed. Saved to: {output_audio_path}")
    
    except Exception as e:
        print(f"Error processing audio: {e}")


In [4]:


# Example Usage
# reduce_background_noise("input.wav", "output_clean.wav")


def convert_video_to_text(video_file):
    """
    Extracts audio from a video file using ffmpeg and converts it to text using the locally downloaded Whisper model.
    """

    # Check if video file exists
    if not os.path.isfile(video_file):
        raise FileNotFoundError(f"Error: The file '{video_file}' was not found.")

    # Define temporary audio output file
    audio_path = "temp_audio.wav"

    
    try:
        # Extract audio using ffmpeg
        command = [
            "ffmpeg", "-i", video_file, "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", audio_path, "-y"
        ]
        subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Load Whisper model
        model = whisper.load_model("large")

        reduce_background_noise(audio_path,audio_path)
        # Transcribe the extracted audio
        result = model.transcribe("temp_audio.wav")

        # Cleanup: Remove temporary audio file
        os.remove(audio_path)

        return result["text"]  # Extract and return transcribed text

    except Exception as e:
        print(f"Error processing video: {e}")
        return None

In [5]:
convert_video_to_text("/Users/akshat/Developer/Python/CODE/Virtual AI Bank Manager/HTML/static/uploads/user_recorded.webm")

Noise reduction completed. Saved to: temp_audio.wav


/Users/akshat/anaconda3/envs/VirtualAI/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


' Hello, my name is Akshat Gowd. I have a salary of 20 lakhs.'

In [6]:
def main():
   # record_audio()
    audio_path = "output.wav" 
    convert_mp3_to_wav(audio_path, "output.wav")
    transcribed_text = convert_audio_to_text("output.wav")
    print(transcribed_text)

In [7]:
#main()

In [9]:
"""import pyaudio
import wave
import threading

# Audio settings
FORMAT = pyaudio.paInt16  # 16-bit audio
CHANNELS = 1  # Mono audio
RATE = 44100  # Sample rate (Hz)
CHUNK = 1024  # Frames per buffer
WAVE_OUTPUT_FILENAME = "recorded_audio.wav"

# Initialize PyAudio
audio = pyaudio.PyAudio()
stream = None
frames = []
recording = False

def start_recording():
    global stream, frames, recording
    if recording:
        print("Recording is already in progress...")
        return

    recording = True
    frames = []  # Clear previous recording

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("🎙️ Recording started...")
    
    def record():
        while recording:
            data = stream.read(CHUNK)
            frames.append(data)

    # Start recording in a separate thread
    threading.Thread(target=record, daemon=True).start()

def stop_recording():
    global stream, frames, recording
    if not recording:
        print("No recording is in progress.")
        return

    recording = False  # Stop recording loop
    stream.stop_stream()
    stream.close()

    # Save the recording to a WAV file
    with wave.open(WAVE_OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"✅ Recording stopped and saved as {WAVE_OUTPUT_FILENAME}")

# Example Usage
start_recording()  # Start recording
input("Press Enter to stop recording...\n")  # Wait for user input
stop_recording()  # Stop recording and save file"""

'import pyaudio\nimport wave\nimport threading\n\n# Audio settings\nFORMAT = pyaudio.paInt16  # 16-bit audio\nCHANNELS = 1  # Mono audio\nRATE = 44100  # Sample rate (Hz)\nCHUNK = 1024  # Frames per buffer\nWAVE_OUTPUT_FILENAME = "recorded_audio.wav"\n\n# Initialize PyAudio\naudio = pyaudio.PyAudio()\nstream = None\nframes = []\nrecording = False\n\ndef start_recording():\n    global stream, frames, recording\n    if recording:\n        print("Recording is already in progress...")\n        return\n\n    recording = True\n    frames = []  # Clear previous recording\n\n    stream = audio.open(format=FORMAT, channels=CHANNELS,\n                        rate=RATE, input=True,\n                        frames_per_buffer=CHUNK)\n\n    print("🎙️ Recording started...")\n\n    def record():\n        while recording:\n            data = stream.read(CHUNK)\n            frames.append(data)\n\n    # Start recording in a separate thread\n    threading.Thread(target=record, daemon=True).start()\n\ndef 